## YUNET

In [ ]:
import cv2
import numpy as np
from datetime import datetime as dt

In [ ]:
cam = cv2.VideoCapture(0)
detector = cv2.FaceDetectorYN.create("yunet_22mar.onnx", "", (320, 320))
i=0
j=0
n_images=500
k=int(n_images/10)
s=dt.now()
while True:
    ret, collection_frame = cam.read()
    if not ret:
        break

    height, width, _ = collection_frame.shape
    detector.setInputSize((width, height))
    _, faces = detector.detect(collection_frame)
    if faces is not None:
        for face in faces:
#             if i%10==0:
#                 j+=1
            x,y,w,h = list(map(int, face[:4]))
            cv2.putText(collection_frame, f'Images Captured: {j}/{k}',(30,30),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255, 127, 0),1,cv2.LINE_AA)
            cv2.rectangle(collection_frame, (x,y), (x+w,y+h), (255,0,0), 2)   

#     i+=1
    cv2.imshow('img', collection_frame)
    if i==n_images:
        break
    if cv2.waitKey(1)==27:
        break

e=dt.now()
t=e-s
print(t.total_seconds())

cam.release()
cv2.destroyAllWindows()

## VGGFACE

In [ ]:
# !pip install keras_vggface
# !pip install tensorflow
# !pip install keras_applications
# !pip install opencv-python
# !pip install opencv-contrib-python

In [ ]:
# change - from keras.engine.topology import get_source_inputs
# to - from keras.utils.layer_utils import get_source_inputs
# in keras vggface, model.py

In [ ]:
# model = VGGFace(model='senet50')
# model = VGGFace(model='resnet50')

In [ ]:
from keras_vggface.vggface import VGGFace
model = VGGFace(model='vgg16')
model = VGGFace()

# PREPROCESSING

In [ ]:
import cv2
import os
import pickle
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

headshots_folder_name = 'dataset'

# dimension of images
image_width = 224
image_height = 224

# for detecting faces
facecascade = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')

# set the directory containing the images
images_dir = os.path.join(".", headshots_folder_name)

current_id = 0
label_ids = {}

# iterates through all the files in each subdirectories
for root, _, files in os.walk(images_dir):
    for file in files:
        if file.endswith("png") or file.endswith("jpg") or file.endswith("jpeg"):
            path = os.path.join(root, file)

            # get the label name (name of the person)
            label = os.path.basename(root).replace(" ", ".").lower()

            # add the label (key) and its number (value)
            if not label in label_ids:
                label_ids[label] = current_id
                current_id += 1

            # load the image
            imgtest = cv2.imread(path, cv2.IMREAD_COLOR)
            image_array = np.array(imgtest, "uint8")

            # get the faces detected in the image
            faces = facecascade.detectMultiScale(imgtest, scaleFactor=1.1, minNeighbors=5)

            # if not exactly 1 face is detected, skip this photo
            if len(faces) != 1:
                print(f'---Photo skipped---\n')
                os.remove(path)
                continue

            # save the detected face(s) and associate
            # them with the label
            for (x_, y_, w, h) in faces:

                # draw the face detected
                face_detect = cv2.rectangle(imgtest,
                        (x_, y_),
                        (x_+w, y_+h),
                        (255, 0, 255), 2)
                
                plt.imshow(face_detect)
                plt.show()

                # resize the detected face to 224x224
                size = (image_width, image_height)

                # detected face region
                roi = image_array[y_: y_ + h, x_: x_ + w]

                # resize the detected head to target size
                resized_image = cv2.resize(roi, size)
                image_array = np.array(resized_image, "uint8")

                # remove the original image
                os.remove(path)

                # replace the image with only the face
                im = Image.fromarray(image_array)
                im.save(path)

# TRANSFER LEARNING

In [ ]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
'./dataset',
target_size=(224,224),
color_mode='rgb',
batch_size=32,
class_mode='categorical',
shuffle=True)

In [ ]:
train_generator.class_indices.values()

NO_CLASSES = len(train_generator.class_indices.values()) #4

In [ ]:
from keras_vggface.vggface import VGGFace

base_model = VGGFace(include_top=True,
            model='vgg16',
            input_shape=(224, 224, 3))

base_model.summary()
print(len(base_model.layers))

In [ ]:
base_model = VGGFace(include_top=False,
                    model='vgg16',
                    input_shape=(224, 224, 3))
base_model.summary()
print(len(base_model.layers))

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)

preds = Dense(NO_CLASSES, activation='softmax')(x)

In [ ]:
model = Model(inputs = base_model.input, outputs = preds)
model.summary()

for layer in model.layers[:19]:
    layer.trainable = False

for layer in model.layers[19:]:
    layer.trainable = True
    
print(len(model.layers))

In [ ]:
model.compile(
        optimizer='Adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'])

In [ ]:
model.fit(train_generator,
          batch_size = 1,
          verbose = 1,
          epochs = 20)

In [ ]:
model.save('emp_face_vgg_model.h5')

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('emp_face_vgg_model.h5', compile=False)

In [ ]:
import pickle

class_dictionary = train_generator.class_indices
class_dictionary = {value:key for key, value in class_dictionary.items()}
print(class_dictionary)

In [ ]:
face_label_filename = 'emp-face-labels.pickle'
with open(face_label_filename, 'wb') as f: 
    pickle.dump(class_dictionary, f)

In [ ]:
import cv2
import os
import pickle
import numpy as np
import pickle
from PIL import Image
import matplotlib.pyplot as plt
# from keras.preprocessing import image
import keras.utils as image
from keras_vggface import utils
from tensorflow.keras.models import load_model

image_width = 224
image_height = 224

face_label_filename = 'emp-face-labels.pickle'
with open(face_label_filename, "rb") as \
    f: class_dictionary = pickle.load(f)

class_list = [value for _, value in class_dictionary.items()]
print(class_list)

## TESTING

In [ ]:
import keras.utils as image
from datetime import datetime as dt

s=dt.now()
facecascade =  cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
model = load_model('emp_face_vgg_model.h5')
test_image_filename = r'C:\Users\seema1\Desktop\facenet,vggface\x.jpeg'
imgtest = cv2.imread(test_image_filename, cv2.IMREAD_COLOR)
image_array = np.array(imgtest, "uint8")
faces = facecascade.detectMultiScale(imgtest, scaleFactor=1.1, minNeighbors=5)

if len(faces) == 1:    
    for (x_, y_, w, h) in faces:
        face_detect = cv2.rectangle(imgtest, (x_, y_), (x_+w, y_+h), (255, 0, 255), 2)
        plt.imshow(face_detect)
        plt.show()

        size = (image_width, image_height)
        roi = image_array[y_: y_ + h, x_: x_ + w]
        resized_image = cv2.resize(roi, size)

        x = image.img_to_array(resized_image)
        x = np.expand_dims(x, axis=0)
        x = utils.preprocess_input(x, version=1)
      
        predicted_prob = model.predict(x)
        print(predicted_prob)
        print(predicted_prob[0].argmax())
        print("Predicted face: " + class_list[predicted_prob[0].argmax()])

e=dt.now()        
print('time: ', (e-s))

## REAL TIME INFERENCE

In [ ]:
from PIL import Image
import numpy as np
import cv2
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model

with tf.device('/GPU:0'):
    image_width = 224
    image_height = 224

    detector = cv2.FaceDetectorYN.create("yunet_22mar.onnx", "", (320, 320))
    model = load_model('emp_face_vgg_model.h5', compile=False)
    with open("emp-face-labels.pickle", 'rb') as f:
        og_labels = pickle.load(f)
        labels = {key:value for key,value in og_labels.items()}

    stream = cv2.VideoCapture(0)
    while(True):
        (grabbed, frame) = stream.read()
        height, width, _ = frame.shape
        detector.setInputSize((width, height))
        _, faces = detector.detect(frame)
        if faces is not None:    
            for face in faces:
                x,y,w,h = list(map(int, face[:4]))

                color = (255, 127, 0)
                stroke = 2
                cv2.rectangle(frame, (x,y), (x+w,y+h), color, stroke)

                roi_rgb = frame[y:y+h, x:x+w]
                size = (image_width, image_height)
                resized_image = cv2.resize(roi_rgb, size)
                image_array = np.array(resized_image, "uint8")
                img = image_array.reshape(1,image_width,image_height,3) 
                img = img.astype('float32')
                img /= 255
                predicted_prob = model.predict(img)

                font = cv2.FONT_HERSHEY_SIMPLEX
                name = labels[predicted_prob[0].argmax()]
                color = (255, 0, 255)
                stroke = 2
                cv2.putText(frame, f'({name})', (x,y-8), font, 1, color, stroke, cv2.LINE_AA)


        cv2.imshow("Image", frame)
        if cv2.waitKey(1)==27:
            break

    stream.release()
    cv2.destroyAllWindows()

### exp

In [ ]:
# with tf.device('/GPU:0'):
image_width = 224
image_height = 224

detector = cv2.FaceDetectorYN.create("yunet_22mar.onnx", "", (320, 320))
model = load_model('model.h5')
with open("labels.pickle", 'rb') as f:
    og_labels = pickle.load(f)
    labels = {key:value for key,value in og_labels.items()}
    
c=0

stream = cv2.VideoCapture(0)
while(True):
    (grabbed, frame) = stream.read()
    if frame is None:
        continue

    height, width, _ = frame.shape
    detector.setInputSize((width, height))
    _, faces = detector.detect(frame)
    if faces is not None:    
        for face in faces:
            x,y,w,h = list(map(int, face[:4]))

            color = (255, 127, 0)
            stroke = 2
            cv2.rectangle(frame, (x,y), (x+w,y+h), color, stroke)

            try:
                roi_rgb = frame[y:y+h, x:x+w]
                size = (image_width, image_height)
                resized_image = cv2.resize(roi_rgb, size)
                image_array = np.array(resized_image, "uint8")
                img = image_array.reshape(1,image_width,image_height,3) 
                img = img.astype('float32')
                img /= 255
                predicted_prob = model.predict(img)
                print(predicted_prob)

                font = cv2.FONT_HERSHEY_SIMPLEX
                print(predicted_prob[0].argmax())
                
                name = labels[predicted_prob[0].argmax()]
                print(name)
                
                color = (255, 0, 255)
                stroke = 2
                cv2.putText(frame, f'({name})', (x,y-8), font, 1, color, stroke, cv2.LINE_AA)
                c+=1

            except:
                continue

    cv2.imshow("Image", frame)
    if c==2 or cv2.waitKey(1)==27:
        break

stream.release()
cv2.destroyAllWindows()